In [42]:
import os 
import pandas as pd
import stanza 
import re

In [4]:
#loading files and parser 
#path= #directory where your file is + file name 
path= f'{(os.getcwd())}/gibbon_decline_volume1_chap21.txt'    #navigate to directory with file you want, replace myfile with the name of your txt file
with open( path, encoding='utf-8', mode='r') as f:
    chapter = f.read()    #replace mychapter with name for variable 

In [5]:
#parsers 
#loading out NER parser 

nlp_ner = stanza.Pipeline(lang='en', processors='tokenize, ner',tokenize_pretokenized=True)


2023-10-25 22:19:36 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2023-10-25 22:19:37 INFO: Loading these models for language: en (English):
| Processor | Package          |
--------------------------------
| tokenize  | combined         |
| ner       | ontonotes_charlm |

2023-10-25 22:19:37 INFO: Using device: cpu
2023-10-25 22:19:37 INFO: Loading: tokenize
2023-10-25 22:19:37 INFO: Loading: ner
2023-10-25 22:19:37 INFO: Done loading processors!


In [8]:
#if you want to pretokenize and split your text by sentence 
tokenized_chapter = chapter.split('. ')
#replace my chapter with your actual variable 

In [9]:
#creating a stanza document 
#can take 3-6 mins depending on ur computer setup 
#if you have pre_tokenized the text 
Docs = [stanza.Document([], text=d) for d in tokenized_chapter]
Out_Docs = nlp_ner(Docs)  
 

In [12]:
#accessing named entities for pre tokenized out_docs 
for doc in Out_Docs:
    print(doc.ents) 

[{
  "text": "The Schism of the Donatists",
  "type": "WORK_OF_ART",
  "start_char": 24,
  "end_char": 51
}, {
  "text": "The Arian Controversy — Athanasius — Distracted state of the Church and the Empire under Constantine",
  "type": "WORK_OF_ART",
  "start_char": 54,
  "end_char": 154
}, {
  "text": "Toleration of Paganism",
  "type": "WORK_OF_ART",
  "start_char": 170,
  "end_char": 192
}]
[{
  "text": "Constantine",
  "type": "PERSON",
  "start_char": 0,
  "end_char": 11
}]
[{
  "text": "Milan,",
  "type": "GPE",
  "start_char": 13,
  "end_char": 19
}, {
  "text": "Roman",
  "type": "NORP",
  "start_char": 93,
  "end_char": 98
}]
[{
  "text": "Catholic",
  "type": "NORP",
  "start_char": 164,
  "end_char": 172
}, {
  "text": "Christianity",
  "type": "NORP",
  "start_char": 227,
  "end_char": 239
}]
[{
  "text": "Constantine",
  "type": "PERSON",
  "start_char": 0,
  "end_char": 11
}]
[]
[{
  "text": "East",
  "type": "LOC",
  "start_char": 94,
  "end_char": 98
}, {
  "text": "Cons

In [48]:
#token.text will show the text of each word, 
#token.ner will show NER, will == 'O' if token is not a named entity

NER_Dict = {}
NER_Dict2 = {}
for i, doc in enumerate(Out_Docs):
    for sent in doc.sentences:
        for token in sent.tokens:
            # adds enumerator to show which sentence each text came from
            NER_Dict[token.text + '@' + str(i)] = token.ner
            NER_Dict2[token.text] = token.ner
            
print(len(NER_Dict.keys()))
print(len(NER_Dict2.keys()))

#create a dictionary (NER_Dict) where the key is the token text and the value is the named entity tag        

15760
4960


In [91]:
#creates a pandas dataframe from dictionary
ner_frame = pd.DataFrame.from_dict(NER_Dict, orient= 'index')

ner_frame = ner_frame.rename(columns={0:'NER_tag'})

# gets root word of each text + enumerator 
ner_frame['token'] = ner_frame.index.str.replace(r'\@.*', '', regex=True)

# converts root to lowercase alphabetical
ner_frame['token'] = ner_frame['token'].str.replace("[^a-zA-Z-]", '', regex=True).str.lower()

# if NER and root are the same, groups all duplicates and counts number of duplicates for that combination 
ner_frame = ner_frame.groupby(ner_frame.columns.tolist(),as_index=False).size()
ner_frame

,NER_tag,token,size
0,B-CARDINAL,four,2
1,B-CARDINAL,hundred,1
2,B-CARDINAL,more,1
3,B-CARDINAL,one,1
4,B-CARDINAL,seven,1
...,...,...,...
4154,S-PERSON,tyre,1
4155,S-PERSON,ursacius,2
4156,S-PERSON,valens,3
4157,S-PERSON,virgin,1
